# Gambler's Ruin

- Player M has 1 dollar and Player N has 2 dollars
- Each play gives one of the players 1 dollar from the other
- Player M wins 2/3 of the games
- They play until one of the two is bankrupt
- *What is the chance that Player M wins?*

____

- We can model this game as a Markov chain
    - Let each state be the balance of Player M
    
- **Note**: all this theory is taken from [here](https://github.com/sinclam2/intro-to-probability-solutions/blob/master/11.2-absorbing-markov-chains/summary-of-chapter.ipynb)

In [1]:
import pandas as pd
import numpy as np

In [2]:
list_states = [0, 1, 2, 3]
df_transition_matrix = pd.DataFrame(np.array([[1, 0, 0, 0],
                                             [1/3, 0, 2/3, 0],
                                             [0, 1/3, 0, 2/3],
                                             [0, 0, 0, 1]]),
                                   index=list_states,
                                   columns=list_states)

In [3]:
df_transition_matrix

,0,1,2,3
0,1.000000,0.000000,0.000000,0.000000
1,0.333333,0.000000,0.666667,0.000000
2,0.000000,0.333333,0.000000,0.666667
3,0.000000,0.000000,0.000000,1.000000


- We can use this matrix to derive the absorption probabilities for states 0 and 3, but first we must convert this matrix to its canonical form i.e. place the transient states first, then the absorbing states

In [8]:
df_transition_matrix = df_transition_matrix.loc[[1,2,0,3], [1,2,0,3]]
df_transition_matrix

,1,2,0,3
1,0.000000,0.666667,0.333333,0.000000
2,0.333333,0.000000,0.000000,0.666667
0,0.000000,0.000000,1.000000,0.000000
3,0.000000,0.000000,0.000000,1.000000


- Now, we define matrices Q and R

In [11]:
Q = df_transition_matrix.loc[[1, 2], [1, 2]].copy()
Q

,1,2
1,0.000000,0.666667
2,0.333333,0.000000


In [12]:
R = df_transition_matrix.loc[[1, 2], [0, 3]]
R

,0,3
1,0.333333,0.000000
2,0.000000,0.666667


- We now must derive $N = (I-Q)^{-1}$

In [16]:
N_array = np.linalg.inv(np.identity(2) - Q.values)
N = pd.DataFrame(N_array)
N

,0,1
0,1.285714,0.857143
1,0.428571,1.285714


- Finally, we know matrix $B = NR$ gives us the absorption probabilities

In [20]:
B = pd.DataFrame(np.matmul(N.values, R.values),
                index=[1, 2],
                columns=[0,3])
B

,0,3
1,0.428571,0.571429
2,0.142857,0.857143


- Therefore, **Player M has a probability of $0.571429\approx 4/7$ of winning**

____

- Let's simulate this game to tie out with our derived probability

In [22]:
def game():
    total = 1
    while True:
        total += np.random.choice([-1, 1, 1])
        if total == 0:
            return 0
        if total == 3:
            return 1

In [28]:
N_trials = 100000
list_results = []

for _ in range(N_trials):
    list_results.append(game())
np.mean(list_results)

0.57129

- Eyyy